In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import random

In [2]:
import numpy as np
from tensorflow.python.keras import backend
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Activation, Dropout

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('C:/Users/Saloni Seth/Downloads/python-project-chatbot-codes/intents.json').read()
intents = json.loads(data_file)


In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [6]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [7]:

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [9]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [10]:

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

Epoch 1/200
47/47 [==============================] - 2s 43ms/step - loss: 2.2310 - acc: 0.1277
Epoch 2/200
47/47 [==============================] - 0s 297us/step - loss: 2.1545 - acc: 0.1702
Epoch 3/200
47/47 [==============================] - 0s 290us/step - loss: 2.1119 - acc: 0.1915
Epoch 4/200
47/47 [==============================] - 0s 284us/step - loss: 1.9585 - acc: 0.3191
Epoch 5/200
47/47 [==============================] - 0s 287us/step - loss: 1.9897 - acc: 0.2553
Epoch 6/200
47/47 [==============================] - 0s 273us/step - loss: 1.8447 - acc: 0.4681
Epoch 7/200
47/47 [==============================] - 0s 297us/step - loss: 1.7111 - acc: 0.6170
Epoch 8/200
47/47 [==============================] - 0s 297us/step - loss: 1.5742 - acc: 0.4894
Epoch 9/200
47/47 [==============================] - 0s 276us/step - loss: 1.4738 - acc: 0.5106
Epoch 10/200
47/47 [==============================] - 0s 279us/step - loss: 1.3295 - acc: 0.6596
Epoch 11/200
47/47 [====================

47/47 [==============================] - 0s 361us/step - loss: 0.0239 - acc: 1.0000
Epoch 85/200
47/47 [==============================] - 0s 365us/step - loss: 0.0439 - acc: 1.0000
Epoch 86/200
47/47 [==============================] - 0s 362us/step - loss: 0.0343 - acc: 1.0000
Epoch 87/200
47/47 [==============================] - 0s 361us/step - loss: 0.0334 - acc: 1.0000
Epoch 88/200
47/47 [==============================] - 0s 361us/step - loss: 0.0558 - acc: 0.9787
Epoch 89/200
47/47 [==============================] - 0s 318us/step - loss: 0.0752 - acc: 0.9787
Epoch 90/200
47/47 [==============================] - 0s 339us/step - loss: 0.0482 - acc: 1.0000
Epoch 91/200
47/47 [==============================] - 0s 361us/step - loss: 0.0419 - acc: 0.9787
Epoch 92/200
47/47 [==============================] - 0s 361us/step - loss: 0.0362 - acc: 1.0000
Epoch 93/200
47/47 [==============================] - 0s 382us/step - loss: 0.0415 - acc: 0.9787
Epoch 94/200
47/47 [=======================

47/47 [==============================] - 0s 435us/step - loss: 0.0512 - acc: 0.9787
Epoch 168/200
47/47 [==============================] - 0s 370us/step - loss: 0.0145 - acc: 1.0000
Epoch 169/200
47/47 [==============================] - 0s 369us/step - loss: 0.0452 - acc: 1.0000
Epoch 170/200
47/47 [==============================] - 0s 301us/step - loss: 0.0198 - acc: 0.9787
Epoch 171/200
47/47 [==============================] - 0s 434us/step - loss: 0.0107 - acc: 1.0000
Epoch 172/200
47/47 [==============================] - 0s 319us/step - loss: 0.0086 - acc: 1.0000
Epoch 173/200
47/47 [==============================] - 0s 295us/step - loss: 0.0175 - acc: 1.0000
Epoch 174/200
47/47 [==============================] - 0s 377us/step - loss: 0.0032 - acc: 1.0000
Epoch 175/200
47/47 [==============================] - 0s 382us/step - loss: 0.0268 - acc: 1.0000
Epoch 176/200
47/47 [==============================] - 0s 488us/step - loss: 0.0270 - acc: 1.0000
Epoch 177/200
47/47 [=============

In [11]:
print("model created")

model created


In [12]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               11392     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 585       
Total params: 20,233
Trainable params: 20,233
Non-trainable params: 0
_________________________________________________________________
